ejecutar este notebook luego de filter_masterfilelist.ipynb para generar un archivo de texto que contiene un listado de todas las entidades reconocidas por gdelt.

In [3]:
import pandas as pd
from multiprocess import Pool
import numpy as np

In [50]:
df = pd.read_csv("english.gkg.oct2020-feb2021.txt")

In [51]:
df

,link,dates
0,http://data.gdeltproject.org/gdeltv2/202010010...,20201001
1,http://data.gdeltproject.org/gdeltv2/202010010...,20201001
2,http://data.gdeltproject.org/gdeltv2/202010010...,20201001
3,http://data.gdeltproject.org/gdeltv2/202010010...,20201001
4,http://data.gdeltproject.org/gdeltv2/202010010...,20201001
...,...,...
11543,http://data.gdeltproject.org/gdeltv2/202102231...,20210223
11544,http://data.gdeltproject.org/gdeltv2/202102231...,20210223
11545,http://data.gdeltproject.org/gdeltv2/202102232...,20210223
11546,http://data.gdeltproject.org/gdeltv2/202102232...,20210223


In [62]:
def extract_entities(link_slice):
    import urllib.request as req
    import zipfile
    import os
    import pandas as pd
    import numpy as np
    entities = set()
#     entities = []
    for url in link_slice:
        try:
            filedata = req.urlopen(url)
            datatowrite = filedata.read()
            filename = url.split('/')[-1]
            with open(f"targetdir/{filename}", 'wb') as f:
                f.write(datatowrite)
            with zipfile.ZipFile(f"targetdir/{filename}","r") as zip_ref:
                zip_ref.extractall("targetdir")
            df = pd.read_csv(f"targetdir/{filename[:-4]}", sep='\t', usecols=[3, 13], header=None, encoding="latin-1")
            df = df.dropna()

            os.remove(f"targetdir/{filename}")
            os.remove(f"targetdir/{filename[:-4]}")

            df[13] = df[13].str.split(';')

#                 df = df[df[3].str.endswith(".cl")].reset_index(drop=True)
            df = df.reset_index(drop=True) # necesario reiniciar el indice antes de concatenar
            enty_list = np.concatenate(df[13])
            entities = entities.union(enty_list)
        except:
            pass
    return entities

In [56]:
def extract_entities_mp():
#     num_of_processes = 32
    data_split = np.array_split(df["link"], 16)
    pool = Pool(16)
    data = pool.map(extract_entities, data_split)
    pool.close()
    pool.join()
    return data

In [8]:
%%time
ents = extract_entities_mp() #8

Wall time: 3min 51s


In [10]:
%%time
ents = extract_entities_mp() #16

Wall time: 3min 47s


In [12]:
%%time
ents = extract_entities_mp() #16 , 16 cores

Wall time: 3min 4s


In [15]:
%%time
ents = extract_entities_mp() #16 , 32 cores

Wall time: 2min 58s


In [17]:
%%time
ents = extract_entities_mp() #32 , 32 cores

Wall time: 2min 59s


In [15]:
%%time
ents = extract_entities_mp() #16 , 16 cores

Wall time: 2min 9s


In [76]:
with open('entities_test.txt', 'w') as f:
    for ent in set.union(*ents): #asegurar que result(ents) no esta vacio
        f.write(f"{ent}\n")